# VGG-16 Model

In [2]:
def vgg_block(input_layer, n_filters, n_conv):
    for _ in range(n_conv):
        input_layer = Conv2D(n_filters, (3, 3), padding = 'same', activation = 'relu')(input_layer)
    input_layer = MaxPooling2D((2, 2), strides = (2, 2))(input_layer)
    return input_layer

In [28]:
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Model
from keras.models import Sequential

def vgg_block(input_layer, n_filters, n_conv):
    for _ in range(n_conv):
        input_layer = Conv2D(n_filters, (3, 3), padding = 'same', activation = 'relu')(input_layer)
    input_layer = MaxPooling2D((2, 2), strides = (2, 2))(input_layer)
    return input_layer

Input = Input(shape = (224, 224, 3))
# vgg block 1
block_1 = vgg_block(Input, 64, 2)
# vgg block 2
block_2 = vgg_block(block_1, 128, 2)
# vgg block 3
block_3 = vgg_block(block_2, 256, 3)
# vgg block 4
block_4 = vgg_block(block_3, 512, 3)
# vgg block 5
block_5 = vgg_block(block_4, 512, 3)
# Flatten
flatten = Flatten()(block_5)
dense_1 = Dense(4096, activation = "relu")(flatten)
dense_2 = Dense(4096, activation = "relu")(dense_1)
dense_3 = Dense(1000, activation = 'softmax')(dense_2)

model = Model(inputs = Input, outputs = dense_3)
model.summary()


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_445 (Conv2D)          (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_446 (Conv2D)          (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_111 (MaxPoolin (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_447 (Conv2D)          (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_448 (Conv2D)          (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_112 (MaxPoolin (None, 56, 56, 128)       0   

In [29]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])

In [ ]:
model.fit()

# Inception Model

In [14]:
def inception_block(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
    # 3x3 conv
    conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(layer_in)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
    # 5x5 conv
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(layer_in)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
    # concatenate filters, 
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out

In [24]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Dense, concatenate, AveragePooling2D, GlobalAveragePooling2D, Flatten
from keras.layers.merge import concatenate
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

def inception_block(input_layer, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(input_layer)
    # 3x3 conv
    conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(input_layer)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
    # 5x5 conv
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(input_layer)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_layer) 
    pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
    # concatenate filters
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out

Input =  Input(shape = (224, 224, 3))

conv_1 = Conv2D(64, (7, 7), padding = 'same', strides = (2, 2), activation = 'relu')(Input)
maxpool_1 = MaxPooling2D((3, 3), padding = 'same', strides = (2, 2))(conv_1)

conv_2 = Conv2D(64, (1 ,1), padding = 'same', strides = (1, 1), activation = 'relu')(maxpool_1)
conv_3 = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu')(conv_2)
maxpool_2 = MaxPooling2D((3, 3), padding='same', strides=(2, 2))(conv_3)

#inception block 1
inception_1 = inception_block(maxpool_2, 64, 96, 128, 16, 32, 32)

#inception block 2
inception_2 = inception_block(inception_1, 128, 128, 192, 32, 96, 64)

maxpool_3 = MaxPooling2D((3, 3), padding = 'same', strides = (2, 2))(inception_2)

#inception block 3
inception_3 = inception_block(maxpool_3, 192, 96, 208, 16, 48, 64)

avgpool_1 = AveragePooling2D((5, 5), strides = 3)(inception_3)
conv_4 = Conv2D(128, (1, 1), padding = 'same', activation = 'relu')(avgpool_1)
flatten_1 = Flatten()(conv_4)

dense_1 = Dense(1024, activation = 'relu')(flatten_1)
dropout_1 = Dropout(0.7)(dense_1)
dense_2 = Dense(10, activation = 'softmax')(dropout_1)

#inception block 4
inception_4 = inception_block(inception_3, 60, 112, 224, 24, 64, 64)

#inception block 5
inception_5 = inception_block(inception_4, 128, 128, 256, 24, 64, 64)

#inception block 6
inception_6 = inception_block(inception_5, 112, 144, 288, 32, 64, 64)

avgpool_2 = AveragePooling2D((5, 5), strides = 3)(inception_6)
conv_5 = Conv2D(128, (1, 1), padding = 'same', activation = 'relu')(avgpool_2)
flatten_2 = Flatten()(conv_5)
dense_3 = Dense(1024, activation = 'relu')(flatten_2)
dropout_2 = Dropout(0.7)(dense_3)
dense_4 = Dense(10, activation = 'softmax')(dropout_2)

#inception block 7
inception_7 = inception_block(inception_6, 256, 160, 320, 32, 128, 128)

maxpool_4 = MaxPooling2D((3, 3), padding = 'same', strides = (2, 2))(inception_7)

inception_8 = inception_block(maxpool_4, 256, 160, 320, 32, 128, 128)

inception_9 = inception_block(inception_8, 384, 192, 384, 48, 128,128)

global_avg_pool = GlobalAveragePooling2D()(inception_9)

dropout_3 = Dropout(0.4)(global_avg_pool)

dense_5 = Dense(10, activation = 'softmax')(dropout_3)

model = Model(Input, [dense_5, dense_2, dense_4])

model.summary()


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_386 (Conv2D)             (None, 112, 112, 64) 9472        input_13[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_98 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_386[0][0]                 
__________________________________________________________________________________________________
conv2d_387 (Conv2D)             (None, 56, 56, 64)   4160        max_pooling2d_98[0][0]           
____________________________________________________________________________________________

In [25]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

In [ ]:
history = model.fit()

# ResNet Model

Two main types of blocks are used in a ResNet, depending mainly on whether the input/output dimensions are same or different. You are going to implement both of them.

In [42]:
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform



def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(F2,kernel_size=(f,f),strides=(1,1),padding='same',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3,(1,1),strides=(1,1),padding='valid',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X


In [43]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters =F1, kernel_size =(1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters =F2, kernel_size =(f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters =F3, kernel_size =(1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters =F3, kernel_size =(1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut]) 
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X


In [44]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2), name = 'avg_pool')(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model


In [46]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 70, 70, 3)    0           input_18[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
history = model.fit()